In [1]:
from pyspark.sql import SparkSession

In [2]:
import dask.dataframe as dd
import datashader as ds
import datashader.transfer_functions as tf

from colorcet import fire

C:\ProgramData\Anaconda34\envs\gpu_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark OpenStreetMap project") \
    .config("spark.some.config.option", "osm-hungary") \
    .getOrCreate()

## Nodes

### spark

In [4]:
hungaryNodes = spark.read.parquet("parquets/hungary-latest.osm.pbf.node.parquet")

In [5]:
print(hungaryNodes.count())
hungaryNodes.show(5)

11272269
+------+-------+-------------+---------+------+--------------------+----+------------------+------------------+
|    id|version|    timestamp|changeset|   uid|            user_sid|tags|          latitude|         longitude|
+------+-------+-------------+---------+------+--------------------+----+------------------+------------------+
|277282|      5|1356326500000| 14386177|740331|    [53 6B 79 6C 69]|  []|        47.5135549|19.047102000000002|
|277285|      9|1315781775000|  9276182| 15565|[6B 65 6D 70 65 6...|  []|        47.5135409|19.058013900000002|
|277286|      7|1315781775000|  9276182| 15565|[6B 65 6D 70 65 6...|  []|        47.5146097|19.058831700000002|
|277290|      3|1295212827000|  6994044| 55843|[66 6C 61 6B 74 6...|  []|        47.5166629|        19.0613926|
|277298|      6|1356326500000| 14386177|740331|    [53 6B 79 6C 69]|  []|47.514618000000006|        19.0435418|
+------+-------+-------------+---------+------+--------------------+----+------------------+---

### dask

In [6]:
df_nodes = dd.io.parquet.read_parquet("parquets/hungary-latest.osm.pbf.node.parquet")
df_nodes = df_nodes.persist()

In [7]:
print(len(df_nodes))
df_nodes.head()

11272269


,id,version,timestamp,changeset,uid,user_sid,latitude,longitude
0,277282,5,1356326500000,14386177,740331,b'Skyli',47.513555,19.047102
1,277285,9,1315781775000,9276182,15565,b'kempelen',47.513541,19.058014
2,277286,7,1315781775000,9276182,15565,b'kempelen',47.514610,19.058832
3,277290,3,1295212827000,6994044,55843,b'flaktack',47.516663,19.061393
4,277298,6,1356326500000,14386177,740331,b'Skyli',47.514618,19.043542


## relations

### spark

In [8]:
hungaryRelations = spark.read.parquet("parquets/hungary-latest.osm.pbf.relation.parquet")
print(hungaryRelations.count())
hungaryRelations.show(5)

52401
+-----+-------+-------------+---------+------+-------------------+--------------------+--------------------+
|   id|version|    timestamp|changeset|   uid|           user_sid|                tags|             members|
+-----+-------+-------------+---------+------+-------------------+--------------------+--------------------+
|11764|      4|1370549745000| 16449377|  8219|[68 75 6E 73 6C 79]|[[name, Képviselő...|[[24025991, outer...|
|11766|      4|1372595148000| 16763856|610781|   [73 7A 61 6C 69]|[[building, yes],...|[[24021187, outer...|
|11767|      3|1372595691000| 16763856|610781|   [73 7A 61 6C 69]|[[building, offic...|[[24024182, outer...|
|11768|      3|1398799751000| 22030053|152074|[62 65 77 65 74 61]|[[type, multipoly...|[[24024210, outer...|
|11772|      7|1445454885000| 34786035|152074|[62 65 77 65 74 61]|[[addr:city, Buda...|[[24026306, inner...|
+-----+-------+-------------+---------+------+-------------------+--------------------+--------------------+
only showing 

### dask

In [9]:
df_relations = dd.io.parquet.read_parquet("parquets/hungary-latest.osm.pbf.relation.parquet")
df_relations = df_relations.persist()

In [10]:
print(len(df_relations))
df_relations.head()

52401


,id,version,timestamp,changeset,uid,user_sid
0,11764,4,1370549745000,16449377,8219,b'hunsly'
1,11766,4,1372595148000,16763856,610781,b'szali'
2,11767,3,1372595691000,16763856,610781,b'szali'
3,11768,3,1398799751000,22030053,152074,b'beweta'
4,11772,7,1445454885000,34786035,152074,b'beweta'


## Ways

### spark

In [11]:
hungaryWays = spark.read.parquet("parquets/hungary-latest.osm.pbf.way.parquet")

In [12]:
print(hungaryWays.count())
hungaryWays.show(5)

1408800
+-------+-------+-------------+---------+-------+--------------------+--------------------+--------------------+
|     id|version|    timestamp|changeset|    uid|            user_sid|                tags|               nodes|
+-------+-------+-------------+---------+-------+--------------------+--------------------+--------------------+
|3175810|      6|1307100504000|  8327728| 247611|[53 7A 50 61 75 6...|[[highway, reside...|[[0, 15231786], [...|
|3175943|      3|1366287135000| 15772683| 408450|[42 C3 A1 74 68 6...|[[highway, reside...|[[0, 15234255], [...|
|3175983|      6|1307094783000|  8327728| 247611|[53 7A 50 61 75 6...|[[highway, reside...|[[0, 15232076], [...|
|3192356|     17|1415790787000| 26732389|1971976|[6E 61 67 79 5F 6...|[[highway, second...|[[0, 1259548666],...|
|3212111|     10|1399325824000| 22155678| 311866| [70 61 70 65 73 7A]|[[highway, second...|[[0, 15475952], [...|
+-------+-------+-------------+---------+-------+--------------------+------------------

### dask

In [13]:
df_ways = dd.io.parquet.read_parquet("parquets/hungary-latest.osm.pbf.way.parquet")
df_ways = df_ways.persist()

In [14]:
print(len(df_ways))
df_ways.head()

1408800


,id,version,timestamp,changeset,uid,user_sid
0,3175810,6,1307100504000,8327728,247611,b'SzPaula'
1,3175943,3,1366287135000,15772683,408450,b'B\xc3\xa1thoryP\xc3\xa9ter'
2,3175983,6,1307094783000,8327728,247611,b'SzPaula'
3,3192356,17,1415790787000,26732389,1971976,b'nagy_balint'
4,3212111,10,1399325824000,22155678,311866,b'papesz'
